# Numerical Exercises 12

In these exercises I will apply neural networks on a well-known classification problem, which consists in correctly understanding (labelling) an handwritten number. To do this I am going to test two different types of network:

- Exercise 12.1: Deep Neural Networks (DNNs), similar to what I used in Exercise 11
- Exercise 12.2: Convolutional Neural Networks (CNNs)

For both types of network I am going to use the MNIST dataset, a collection of $70000$ handwritten digits saved in a $28\times 28$ pixel grayscale ($255$ gradations of the gray color) images. Once I obtain a working network, in Exercise 12.3 I will use it to label my very own handwritten digits. 

As in Lab11, since everything has been written in python, my comments and the obtained results are noted in the same notebook as the code they refer to.